# Trajectory of a Projectile: The Parabola
This tutorial demonstrates how to simulate a simple parabolic trajectory using the SOFA (Simulation Open Framework Architecture) physics engine. 

### Learning Objectives
- Initialize a SOFA simulation in a Python environment.
- Create a basic Scene Graph with a root node.
- Add essential components: solvers, state containers, and mass.
- Run a simulation loop and retrieve data for visualization.

In this example, we will simulate a single particle (a point mass) subjected to constant gravity, starting with an initial velocity. According to Newtonian mechanics, this results in a parabolic path.

## 1. Environment Setup
First, we must import the `Sofa` and `SofaRuntime` modules. `SofaRuntime.init()` is required to initialize the SOFA runtime environment.

In [ ]:
import Sofa
import SofaRuntime
SofaRuntime.init()

## 2. Creating the Scene Graph
In SOFA, simulations are organized in a tree structure called a **Scene Graph**. Every component is attached to a `Node`. We start by creating the `root` node.

We also define the gravity vector here. In this case, we use a standard Earth gravity of $-9.81 m/s^2$ along the Y-axis.

In [ ]:
root = Sofa.Core.Node("root")
root.gravity.value = [0, -9.81, 0]

## 3. Simulation Components
To make the simulation run, we need to add several functional components to our scene.

### Animation Loop
The `AnimationLoop` is responsible for triggering the simulation steps. The `DefaultAnimationLoop` is the simplest one, handling time-stepping linearly.

In [ ]:
root.addObject("DefaultAnimationLoop")

### ODE Solver
An Ordinary Differential Equation (ODE) solver is required to integrate the laws of motion over time. Here, we use the `EulerExplicitSolver`, which is a first-order integrator.

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.ODESolver.Forward")
root.addObject("EulerExplicitSolver")

### State Container (MechanicalObject)
The `MechanicalObject` stores the degrees of freedom (DOFs) of our object—in this case, its position and velocity. We use `template="Vec2"` because we are working in a 2D plane.

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.StateContainer")
initial_position = [0.0, 0.0]
initial_velocity = [1.0, 1.0]
root.addObject("MechanicalObject", template="Vec2", name="particle", 
    position=[initial_position], velocity=[initial_velocity])

### Mass
To respond to forces like gravity, the particle needs a mass. `UniformMass` assigns a constant mass to each vertex in the `MechanicalObject`.

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.Mass")
root.addObject("UniformMass", template="Vec2", name="mass", vertexMass=1.0)

## 4. Running the Simulation
Now that the scene is defined, we initialize it and run the animation loop for several iterations.

In [ ]:
Sofa.Simulation.initRoot(root)

We will store the (x, y) coordinates at each time step to plot the trajectory later.

In [ ]:
x = []
y = []

def retrieve_position():
    """Helper function to extract the current position of the particle."""
    # .value returns a numpy-like array of positions
    position = root.particle.position.value
    assert len(position) == 1
    point = position[0]
    x.append(point[0])
    y.append(point[1])

# Record the initial position
retrieve_position()

# Run 20 simulation steps
for iteration in range(20):
    # Advance the simulation by one time step (dt)
    Sofa.Simulation.animate(root, root.dt.value)
    retrieve_position()

## 5. Visualization
We use `bokeh` to visualize the resulting trajectory.

In [ ]:
from bokeh.plotting import figure, output_notebook, show
output_notebook() # Enable inline plotting for Jupyter Notebook

p = figure(title="Parabolic Trajectory in SOFA", x_axis_label='x (m)', y_axis_label='y (m)')
p.line(x, y, line_width=2, line_color="blue", legend_label="Trajectory")
p.scatter(x, y, size=10, color="orange", legend_label="Time Steps")
p.legend.location = "top_left"
show(p)

### Analysis
The plot shows the classic parabolic trajectory of a projectile. 
- The **blue line** represents the interpolated path.
- The **orange dots** represent the discrete positions recorded at each simulation time step. 

Since we used a simple `EulerExplicitSolver` and a constant gravity field, the particle follows the expected kinematic equations:
$y(x) = y_0 + \tan(\theta)x - \frac{g}{2v_0^2\cos^2(\theta)}x^2$